In [1]:
import pandas as pd
import numpy as np

from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import LSTM, Input, InputLayer, Dropout, Dense, Flatten, Embedding, Add, Concatenate
from tensorflow.keras import Sequential
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping
import tensorflow as tf

# from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.utils import class_weight

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from tensorflow.keras.preprocessing.text import Tokenizer
from sklearn.preprocessing import LabelEncoder
from keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.optimizers import RMSprop

import matplotlib.pyplot as plt

from sklearn.metrics import precision_recall_fscore_support

## Own code 
import import_ipynb
import metadata_options
import models_nn

importing Jupyter notebook from metadata_options.ipynb
importing Jupyter notebook from models_nn.ipynb


In [2]:
# Data with Throughput & WorkTime
df = pd.read_csv('./data/kokil dec 6 reprepare/conf_pc_worker_sem.csv')
df = df.dropna()

In [3]:
######################################
## Weighted Onehot Encoding options ##
######################################

##############
# Throughput #
##############
# TP1: weighted by 1 average per set of OHE, i.e. (a, b, c, d) -> (w*a, w*b, w*c, w*d)
# TP2: weighted by 1 linear variance per set of OHE, i.e. (a, b, c, d) -> (w*a, w*b, w*c, w*d)
# TP3 + k: weighted by 1 inverted k-power U-shaped variance per set of OHE, i.e. (a, b, c, d) -> (w*a, w*b, w*c, w*d)
# TP4 + k: weighted by 1 upright k-power U-shaped variance per set of OHE, i.e. (a, b, c, d) -> (w*a, w*b, w*c, w*d)
# (For TP3 & TP4, k=1 results in V-shaped variance, and as k>1 increases, sides will curve into U-shaped variance)

############
# Worktime #
############
# WT1: weighted by 1 average per set of OHE, i.e. (a, b, c, d) -> (w*a, w*b, w*c, w*d)
# WT2: weighted by 1 linear variance per set of OHE, i.e. (a, b, c, d) -> (w*a, w*b, w*c, w*d)

################
# PC agreement #
################
# PC1: weighted by 1 average per set of OHE, i.e. (a, b, c, d) -> (w*a, w*b, w*c, w*d)
# PC2: weighted by 1 linear variance per set of OHE, i.e. (a, b, c, d) -> (w*a, w*b, w*c, w*d)
# PC3: weighted by 1 PC agreement weight per annotation in each OHE, i.e. (a, b, c, d) -> (w1*a, w2*b, w3*c, w4*d)

#####################
# Input text length #
#####################
# TL1: weighted by 1 normalised number of characters per set of OHE, i.e. (a, b, c, d) -> (w*a, w*b, w*c, w*d)
# TL2: weighted by 1 normalised number of words per set of OHE, i.e. (a, b, c, d) -> (w*a, w*b, w*c, w*d)

###################
# Special Options #
###################
# SP1: weighted by average of TP1 and TP2 per set of OHE, i.e. (a, b, c, d) -> (w*a, w*b, w*c, w*d)
# SP2: weighted by average of WT1 and WT2 per set of OHE, i.e. (a, b, c, d) -> (w*a, w*b, w*c, w*d)
# SP3: weighted by average of PC1 and PC2 per set of OHE, i.e. (a, b, c, d) -> (w*a, w*b, w*c, w*d)
# RAND_UNI: weighted by 1 uniformly distributed random number between 0 to 1 per set of OHE, i.e. (a, b, c, d) -> (w*a, w*b, w*c, w*d)
# RAND_NORM: weighted by 1 normally distributed random number between 0 to 1 per set of OHE, i.e. (a, b, c, d) -> (w*a, w*b, w*c, w*d)

# Select 1 option from each of the few variants above, e.g. TP2, WT1, PC3, TL1, SP3, and input into function
# set_OHE_pipeline_options. If not selecting TP3 or TP4, input k (option_k) will be ignored. After
# editing the options, run the entire notebook for results accordingly.

# Edit option choices here
throughput_option = 'TP1'
worktime_option = 'WT1'
pc_agreement_option = 'PC1'
textlength_option = 'TL1'
special_option = 'SP1'
k_option_for_tp = 3

df_throughput, df_worktime, df_agreement, df_textlength, df_special = metadata_options.set_OHE_pipeline_options(df, throughput_option, worktime_option, pc_agreement_option, textlength_option, special_option, k_option_for_tp)

TP1: weighted by 1 average per set of OHE, i.e. (a, b, c, d) -> (w*a, w*b, w*c, w*d)
WT1: weighted by 1 average per set of OHE, i.e. (a, b, c, d) -> (w*a, w*b, w*c, w*d)
PC1: weighted by 1 average per set of OHE, i.e. (a, b, c, d) -> (w*a, w*b, w*c, w*d)
TL1: weighted by 1 normalised number of characters per set of OHE, i.e. (a, b, c, d) -> (w*a, w*b, w*c, w*d)
SP1: weighted by average of TP1 and TP2 per set of OHE, i.e. (a, b, c, d) -> (w*a, w*b, w*c, w*d)


In [4]:
######################################
## Model Options ##
######################################
# options: lstm, cnn, lstm-attn

model_name = 'lstm-attn'
models_nn.MODEL_NAME = model_name

In [5]:
df.head()

,Input.sentence_id,HITId,Input.convo_id,Input.train_test_val,Input.msg_id,Input.timestamp,Input.full_text,Input.speaker,Input.reply_to,Input.speaker_intention,...,prt,punct,purpcl,quantmod,rcmod,rel,root,tmod,xcomp,xsubj
5,11,3MG8450X2OASXZ0WO9O5AH70GU3UPA,Game1-italy-germany,Train,Game1-italy-germany-3,87,It seems like there are a lot of ways that cou...,germany-Game1,Game1-italy-germany-2,Truth,...,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
6,12,38G0E1M85M552JXSALX4G9WI2I6UVX,Game1-italy-germany,Train,Game1-italy-germany-7,117,"Sorry Italy I've been away doing, um, German t...",germany-Game1,Game1-italy-germany-6,Truth,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
7,14,3HYV4299H0WQ2B4TCS7PKDQ75WHE81,Game1-italy-germany,Train,Game1-italy-germany-8,119,"I don't think I'm ready to go for that idea, h...",germany-Game1,Game1-italy-germany-7,Truth,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
8,15,3XU9MCX6VOC4P079IHIO9TCNYLGR2P,Game1-italy-germany,Train,Game1-italy-germany-9,121,I am pretty conflicted about whether to guess ...,italy-Game1,Game1-italy-germany-8,Truth,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
9,16,3FVBZG9CLJEK4WQS7P2GC1H2EEQH0Q,Game1-italy-germany,Train,Game1-italy-germany-9,121,I am going to take it literally and say even ...,italy-Game1,Game1-italy-germany-8,Truth,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,2.0,1.0


In [6]:
# # Old train_test_split code
# train, test, indices_train, indices_test = train_test_split(df, indices, test_size=0.2)

# New train_test_split using Stratified Shaffled Splits
y = df["Input.deception_quadrant"].copy()
X = df.drop(["Input.deception_quadrant"], axis=1)
sss = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=0)
splits_generator = sss.split(X, y)

for train_idx, test_idx in splits_generator:
    indices_train = train_idx
    indices_test = test_idx

train = df.take(indices_train)
test = df.take(indices_test)

In [7]:
print(df.shape, train.shape, test.shape)

(11366, 862) (9092, 862) (2274, 862)


In [8]:
# check scikit-learn version
import sklearn
print('sklearn: %s' % sklearn.__version__)

sklearn: 0.23.2


In [9]:
new_deception_train = train["Input.deception_quadrant"].copy()
new_deception_train['Input.deception_quadrant'] = train["Input.deception_quadrant"].apply(lambda x : 1 if x == "Straightforward" else 0)
y_train_deception = new_deception_train['Input.deception_quadrant'].to_numpy()
deception_class_weights = class_weight.compute_class_weight('balanced',
                                                 np.unique(y_train_deception),
                                                 y_train_deception)
deception_class_weight_dict = dict(enumerate(deception_class_weights))

y_train_rapport = train['Answer.3rapport.yes_label'].tolist()
rapport_class_weights = class_weight.compute_class_weight('balanced',
                                                 np.unique(y_train_rapport),
                                                 y_train_rapport)
rapport_class_weight_dict = dict(enumerate(rapport_class_weights))

y_train_share_information = train['Answer.4shareinformation.yes_label'].tolist()
share_info_class_weights = class_weight.compute_class_weight('balanced',
                                                 np.unique(y_train_share_information),
                                                 y_train_share_information)
share_info_class_weight_dict = dict(enumerate(share_info_class_weights))

y_train_reasoning = train['Answer.2reasoning.yes_label'].tolist()
reasoning_class_weights = class_weight.compute_class_weight('balanced',
                                                 np.unique(y_train_reasoning),
                                                 y_train_reasoning)
reasoning_class_weight_dict = dict(enumerate(reasoning_class_weights))

y_train_gamemove = train['Answer.1gamemove.yes_label'].tolist()
gamemove_class_weights = class_weight.compute_class_weight('balanced',
                                                 np.unique(y_train_gamemove),
                                                 y_train_gamemove)
gamemove_class_weight_dict = dict(enumerate(gamemove_class_weights))

/home/kaiyuan/anaconda3/envs/TF2/lib/python3.6/site-packages/sklearn/utils/validation.py:70: FutureWarning: Pass classes=[0 1], y=[1 1 1 ... 1 1 1] as keyword args. From version 0.25 passing these as positional arguments will result in an error
  FutureWarning)
/home/kaiyuan/anaconda3/envs/TF2/lib/python3.6/site-packages/sklearn/utils/validation.py:70: FutureWarning: Pass classes=[0. 1.], y=[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 

In [10]:
print("Class weights")
print("Deception: {} \nRapport: {} \nShare Information: {} \nReasoning: {} \nGamemove: {}".format(deception_class_weight_dict,
                                                                                                  rapport_class_weight_dict,
                                                                                                  share_info_class_weight_dict,
                                                                                                  reasoning_class_weight_dict,
                                                                                                  gamemove_class_weight_dict))

Class weights
Deception: {0: 10.192825112107624, 1: 0.525792273883877} 
Rapport: {0: 3.582348305752561, 1: 0.5811069922024799} 
Share Information: {0: 3.117969821673525, 1: 0.59549384333246} 
Reasoning: {0: 2.9596354166666665, 1: 0.6016410799364743} 
Gamemove: {0: 7.048062015503876, 1: 0.5381792352314431}


In [11]:
X_train_col = train['Input.full_text']

new_deception_test = test["Input.deception_quadrant"].copy()
new_deception_test['Input.deception_quadrant'] = test["Input.deception_quadrant"].apply(lambda x : 1 if x == "Straightforward" else 0)
y_test_deception = new_deception_test['Input.deception_quadrant'].tolist()
y_test_rapport = test['Answer.3rapport.yes_label'].tolist()
y_test_share_information = test['Answer.4shareinformation.yes_label'].tolist()
y_test_reasoning = test['Answer.2reasoning.yes_label'].tolist()
y_test_gamemove = test['Answer.1gamemove.yes_label'].tolist()

X_test_col = test['Input.full_text']

In [12]:
le = LabelEncoder()

# def label_preprocessing(y_data, label_encoder):
#     out = label_encoder.fit_transform(y_data).reshape(-1,1)
#     return out

y_train_deception = le.fit_transform(y_train_deception)
y_train_deception = y_train_deception.reshape(-1,1)

y_train_rapport = le.fit_transform(y_train_rapport)
y_train_rapport = y_train_rapport.reshape(-1,1)

y_train_share_information = le.fit_transform(y_train_share_information)
y_train_share_information = y_train_share_information.reshape(-1,1)

y_train_reasoning = le.fit_transform(y_train_reasoning)
y_train_reasoning = y_train_reasoning.reshape(-1,1)

y_train_gamemove = le.fit_transform(y_train_gamemove)
y_train_gamemove = y_train_gamemove.reshape(-1,1)

y_train_deception = le.fit_transform(y_train_deception)
y_train_deception = y_train_deception.reshape(-1,1)

y_test_rapport = le.fit_transform(y_test_rapport)
y_test_rapport = y_test_rapport.reshape(-1,1)

y_test_share_information = le.fit_transform(y_test_share_information)
y_test_share_information = y_test_share_information.reshape(-1,1)

y_test_reasoning = le.fit_transform(y_test_reasoning)
y_test_reasoning = y_test_reasoning.reshape(-1,1)

y_test_gamemove = le.fit_transform(y_test_gamemove)
y_test_gamemove = y_test_gamemove.reshape(-1,1)

y_test_deception = le.fit_transform(y_test_deception)
y_test_deception = y_test_deception.reshape(-1,1)

y_test_rapport = np.asarray(y_test_rapport)
y_train_rapport = np.asarray(y_train_rapport)

/home/kaiyuan/anaconda3/envs/TF2/lib/python3.6/site-packages/sklearn/utils/validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


In [13]:
max_words = 1000
max_len = 220

tok = Tokenizer(num_words=max_words)

tok.fit_on_texts(X_train_col)
X_train_sequences = tok.texts_to_sequences(X_train_col)
X_train = pad_sequences(X_train_sequences, maxlen=max_len)

X_test_sequences = tok.texts_to_sequences(X_test_col)
X_test = pad_sequences(X_test_sequences, maxlen=max_len)

### Construct individual  models

In [14]:
# Rapport model
rapport_model = models_nn.create_nn_model()
rapport_model.compile(loss='binary_crossentropy',optimizer=RMSprop(),metrics=['accuracy', models_nn.f1_m, 
                                                                              models_nn.recall_m, models_nn.precision_m])
rapport_model.fit(X_train,y_train_rapport,
                  batch_size=128,
                  epochs=50,
                  validation_data=(X_test, y_test_rapport), 
#                   callbacks=[models_nn.early_stop],
                  class_weight=rapport_class_weight_dict)

Epoch 1/50
72/72 [==============================] - 2s 29ms/step - loss: 0.6932 - accuracy: 0.8379 - f1_m: 0.8948 - recall_m: 0.9721 - precision_m: 0.8306 - val_loss: 0.6947 - val_accuracy: 0.1288 - val_f1_m: 0.0000e+00 - val_recall_m: 0.0000e+00 - val_precision_m: 0.0000e+00
Epoch 2/50
72/72 [==============================] - 2s 24ms/step - loss: 0.6931 - accuracy: 0.4908 - f1_m: 0.5138 - recall_m: 0.4965 - precision_m: 0.6414 - val_loss: 0.6958 - val_accuracy: 0.2058 - val_f1_m: 0.1796 - val_recall_m: 0.1004 - val_precision_m: 0.8946
Epoch 3/50
72/72 [==============================] - 2s 25ms/step - loss: 0.6918 - accuracy: 0.4061 - f1_m: 0.4599 - recall_m: 0.3532 - precision_m: 0.8441 - val_loss: 0.6874 - val_accuracy: 0.3975 - val_f1_m: 0.5053 - val_recall_m: 0.3552 - val_precision_m: 0.8829
Epoch 4/50
72/72 [==============================] - 2s 24ms/step - loss: 0.6843 - accuracy: 0.4816 - f1_m: 0.5672 - recall_m: 0.4462 - precision_m: 0.8871 - val_loss: 0.8034 - val_accuracy: 0.1

Epoch 32/50
72/72 [==============================] - 2s 23ms/step - loss: 0.4818 - accuracy: 0.7369 - f1_m: 0.8227 - recall_m: 0.7247 - precision_m: 0.9598 - val_loss: 0.8561 - val_accuracy: 0.5070 - val_f1_m: 0.6439 - val_recall_m: 0.5128 - val_precision_m: 0.8674
Epoch 33/50
72/72 [==============================] - 2s 25ms/step - loss: 0.4675 - accuracy: 0.7405 - f1_m: 0.8224 - recall_m: 0.7227 - precision_m: 0.9645 - val_loss: 0.5774 - val_accuracy: 0.7454 - val_f1_m: 0.8507 - val_recall_m: 0.8380 - val_precision_m: 0.8656
Epoch 34/50
72/72 [==============================] - 2s 25ms/step - loss: 0.4635 - accuracy: 0.7464 - f1_m: 0.8276 - recall_m: 0.7304 - precision_m: 0.9638 - val_loss: 0.6162 - val_accuracy: 0.7115 - val_f1_m: 0.8254 - val_recall_m: 0.7874 - val_precision_m: 0.8690
Epoch 35/50
72/72 [==============================] - 2s 25ms/step - loss: 0.4547 - accuracy: 0.7604 - f1_m: 0.8390 - recall_m: 0.7478 - precision_m: 0.9614 - val_loss: 0.9697 - val_accuracy: 0.4529 - va

In [15]:
rapport_pred = rapport_model.predict(X_train)
rapport_pred_test = rapport_model.predict(X_test)

rapport_pred_test_round = rapport_pred_test.round()
precision_recall_fscore_support(y_test_rapport, rapport_pred_test_round, average='macro')

(0.48555469456283096, 0.47687588403829556, 0.47311624243200223, None)

In [16]:
# Game move model
gamemove_model = models_nn.create_nn_model()
gamemove_model.compile(loss='binary_crossentropy',optimizer=RMSprop(),metrics=['accuracy', models_nn.f1_m, 
                                                                               models_nn.recall_m, models_nn.precision_m])
gamemove_model.fit(X_train,
                   y_train_gamemove,
                   batch_size=128,
                   epochs=50,
                   validation_data=(X_test, y_test_gamemove), 
#                    callbacks=[models_nn.early_stop],
                   class_weight=gamemove_class_weight_dict)

Epoch 1/50
72/72 [==============================] - 2s 29ms/step - loss: 0.6932 - accuracy: 0.8709 - f1_m: 0.9052 - recall_m: 0.9343 - precision_m: 0.8910 - val_loss: 0.6897 - val_accuracy: 0.9376 - val_f1_m: 0.9675 - val_recall_m: 1.0000 - val_precision_m: 0.9372
Epoch 2/50
72/72 [==============================] - 2s 25ms/step - loss: 0.6937 - accuracy: 0.8271 - f1_m: 0.8604 - recall_m: 0.8873 - precision_m: 0.8510 - val_loss: 0.6847 - val_accuracy: 0.9376 - val_f1_m: 0.9675 - val_recall_m: 1.0000 - val_precision_m: 0.9372
Epoch 3/50
72/72 [==============================] - 2s 25ms/step - loss: 0.6909 - accuracy: 0.4706 - f1_m: 0.5645 - recall_m: 0.4600 - precision_m: 0.9449 - val_loss: 0.6375 - val_accuracy: 0.5963 - val_f1_m: 0.7357 - val_recall_m: 0.6019 - val_precision_m: 0.9481
Epoch 4/50
72/72 [==============================] - 2s 25ms/step - loss: 0.6808 - accuracy: 0.4287 - f1_m: 0.5526 - recall_m: 0.4038 - precision_m: 0.9557 - val_loss: 0.8604 - val_accuracy: 0.0721 - val_f1

Epoch 32/50
72/72 [==============================] - 2s 25ms/step - loss: 0.3677 - accuracy: 0.7916 - f1_m: 0.8717 - recall_m: 0.7816 - precision_m: 0.9910 - val_loss: 0.7677 - val_accuracy: 0.6451 - val_f1_m: 0.7774 - val_recall_m: 0.6627 - val_precision_m: 0.9433
Epoch 33/50
72/72 [==============================] - 2s 25ms/step - loss: 0.3637 - accuracy: 0.7967 - f1_m: 0.8765 - recall_m: 0.7911 - precision_m: 0.9879 - val_loss: 1.2637 - val_accuracy: 0.3628 - val_f1_m: 0.5002 - val_recall_m: 0.3415 - val_precision_m: 0.9412
Epoch 34/50
72/72 [==============================] - 2s 25ms/step - loss: 0.3589 - accuracy: 0.7915 - f1_m: 0.8683 - recall_m: 0.7772 - precision_m: 0.9930 - val_loss: 0.4951 - val_accuracy: 0.7498 - val_f1_m: 0.8544 - val_recall_m: 0.7842 - val_precision_m: 0.9402
Epoch 35/50
72/72 [==============================] - 2s 24ms/step - loss: 0.3520 - accuracy: 0.8077 - f1_m: 0.8861 - recall_m: 0.8035 - precision_m: 0.9911 - val_loss: 0.6324 - val_accuracy: 0.7274 - va

In [17]:
gamemove_pred = gamemove_model.predict(X_train)
gamemove_pred_test = gamemove_model.predict(X_test)

gamemove_pred_test_round = gamemove_pred_test.round()
precision_recall_fscore_support(y_test_gamemove, gamemove_pred_test_round, average='macro')

(0.5063325790203664, 0.516046560790635, 0.49230886040251715, None)

In [18]:
# Reasoning model
reasoning_model = models_nn.create_nn_model()
reasoning_model.compile(loss='binary_crossentropy',optimizer=RMSprop(),metrics=['accuracy', models_nn.f1_m, models_nn.recall_m, models_nn.precision_m])
reasoning_model.fit(X_train,
                    y_train_reasoning,
                    batch_size=128,
                    epochs=50,
                    validation_data=(X_test, y_test_reasoning), 
#                     callbacks=[models_nn.early_stop],
                    class_weight=reasoning_class_weight_dict)

Epoch 1/50
72/72 [==============================] - 2s 30ms/step - loss: 0.6932 - accuracy: 0.5485 - f1_m: 0.5389 - recall_m: 0.5863 - precision_m: 0.5221 - val_loss: 0.6912 - val_accuracy: 0.8219 - val_f1_m: 0.9011 - val_recall_m: 1.0000 - val_precision_m: 0.8212
Epoch 2/50
72/72 [==============================] - 2s 25ms/step - loss: 0.6932 - accuracy: 0.8153 - f1_m: 0.8927 - recall_m: 0.9783 - precision_m: 0.8352 - val_loss: 0.6869 - val_accuracy: 0.8219 - val_f1_m: 0.9011 - val_recall_m: 1.0000 - val_precision_m: 0.8212
Epoch 3/50
72/72 [==============================] - 2s 25ms/step - loss: 0.6924 - accuracy: 0.6276 - f1_m: 0.7020 - recall_m: 0.6920 - precision_m: 0.8472 - val_loss: 0.6973 - val_accuracy: 0.3646 - val_f1_m: 0.4098 - val_recall_m: 0.2707 - val_precision_m: 0.8612
Epoch 4/50
72/72 [==============================] - 2s 25ms/step - loss: 0.6876 - accuracy: 0.4227 - f1_m: 0.4970 - recall_m: 0.3619 - precision_m: 0.8730 - val_loss: 0.6866 - val_accuracy: 0.4371 - val_f1

Epoch 32/50
72/72 [==============================] - 2s 25ms/step - loss: 0.5345 - accuracy: 0.6443 - f1_m: 0.7290 - recall_m: 0.5991 - precision_m: 0.9546 - val_loss: 0.8166 - val_accuracy: 0.3953 - val_f1_m: 0.4737 - val_recall_m: 0.3350 - val_precision_m: 0.8238
Epoch 33/50
72/72 [==============================] - 2s 26ms/step - loss: 0.5288 - accuracy: 0.6466 - f1_m: 0.7370 - recall_m: 0.6076 - precision_m: 0.9582 - val_loss: 0.9659 - val_accuracy: 0.4002 - val_f1_m: 0.4799 - val_recall_m: 0.3410 - val_precision_m: 0.8287
Epoch 34/50
72/72 [==============================] - 2s 26ms/step - loss: 0.5208 - accuracy: 0.6562 - f1_m: 0.7342 - recall_m: 0.6067 - precision_m: 0.9450 - val_loss: 0.6316 - val_accuracy: 0.6240 - val_f1_m: 0.7509 - val_recall_m: 0.6960 - val_precision_m: 0.8182
Epoch 35/50
72/72 [==============================] - 2s 25ms/step - loss: 0.5172 - accuracy: 0.6683 - f1_m: 0.7502 - recall_m: 0.6280 - precision_m: 0.9500 - val_loss: 0.6191 - val_accuracy: 0.6750 - va

In [19]:
reasoning_pred = reasoning_model.predict(X_train)
reasoning_pred_test = reasoning_model.predict(X_test)

reasoning_pred_test_round = reasoning_pred_test.round()
precision_recall_fscore_support(y_test_reasoning, reasoning_pred_test_round, average='macro')

(0.503475901279235, 0.5047975744604958, 0.4964867737364194, None)

In [20]:
# Share Information model
shareinfo_model = models_nn.create_nn_model()
shareinfo_model.compile(loss='binary_crossentropy',optimizer=RMSprop(),metrics=['accuracy', models_nn.f1_m, models_nn.recall_m, models_nn.precision_m])
shareinfo_model.fit(X_train,
                    y_train_share_information,
                    batch_size=128,
                    epochs=50,
                    validation_data=(X_test, y_test_share_information), 
#                     callbacks=[models_nn.early_stop],
                    class_weight=share_info_class_weight_dict)

Epoch 1/50
72/72 [==============================] - 2s 30ms/step - loss: 0.6932 - accuracy: 0.5613 - f1_m: 0.5330 - recall_m: 0.5839 - precision_m: 0.5101 - val_loss: 0.6962 - val_accuracy: 0.1557 - val_f1_m: 0.0000e+00 - val_recall_m: 0.0000e+00 - val_precision_m: 0.0000e+00
Epoch 2/50
72/72 [==============================] - 2s 24ms/step - loss: 0.6932 - accuracy: 0.2525 - f1_m: 0.1603 - recall_m: 0.1478 - precision_m: 0.2553 - val_loss: 0.6900 - val_accuracy: 0.8443 - val_f1_m: 0.9147 - val_recall_m: 1.0000 - val_precision_m: 0.8436
Epoch 3/50
72/72 [==============================] - 2s 25ms/step - loss: 0.6911 - accuracy: 0.5499 - f1_m: 0.6241 - recall_m: 0.5594 - precision_m: 0.8482 - val_loss: 0.6672 - val_accuracy: 0.5928 - val_f1_m: 0.7197 - val_recall_m: 0.6220 - val_precision_m: 0.8578
Epoch 4/50
72/72 [==============================] - 2s 24ms/step - loss: 0.6807 - accuracy: 0.5016 - f1_m: 0.5933 - recall_m: 0.4676 - precision_m: 0.8676 - val_loss: 0.7432 - val_accuracy: 0.3

Epoch 32/50
72/72 [==============================] - 2s 26ms/step - loss: 0.4358 - accuracy: 0.7288 - f1_m: 0.8107 - recall_m: 0.7007 - precision_m: 0.9696 - val_loss: 0.7274 - val_accuracy: 0.6785 - val_f1_m: 0.7974 - val_recall_m: 0.7551 - val_precision_m: 0.8474
Epoch 33/50
72/72 [==============================] - 2s 25ms/step - loss: 0.4265 - accuracy: 0.7397 - f1_m: 0.8080 - recall_m: 0.7026 - precision_m: 0.9564 - val_loss: 0.8215 - val_accuracy: 0.6618 - val_f1_m: 0.7842 - val_recall_m: 0.7312 - val_precision_m: 0.8480
Epoch 34/50
72/72 [==============================] - 2s 25ms/step - loss: 0.4160 - accuracy: 0.7467 - f1_m: 0.8273 - recall_m: 0.7239 - precision_m: 0.9711 - val_loss: 1.1507 - val_accuracy: 0.4446 - val_f1_m: 0.5556 - val_recall_m: 0.4139 - val_precision_m: 0.8530
Epoch 35/50
72/72 [==============================] - 2s 25ms/step - loss: 0.4119 - accuracy: 0.7519 - f1_m: 0.8312 - recall_m: 0.7296 - precision_m: 0.9726 - val_loss: 0.9580 - val_accuracy: 0.5079 - va

In [21]:
shareinfo_pred = shareinfo_model.predict(X_train)
shareinfo_pred_test = shareinfo_model.predict(X_test)

shareinfo_pred_test_round = shareinfo_pred_test.round()
precision_recall_fscore_support(y_test_share_information, shareinfo_pred_test_round, average='macro')

(0.5085412207542992, 0.513621115819209, 0.49556733001676634, None)

In [22]:
# Deception model
deception_model = models_nn.create_nn_model()
deception_model.compile(loss='binary_crossentropy',optimizer=RMSprop(),metrics=['accuracy', models_nn.f1_m, models_nn.recall_m, models_nn.precision_m])
deception_model.fit(X_train,
                    y_train_deception,
                    epochs=50,
                    validation_data=(X_test, y_test_deception), 
#                     callbacks=[models_nn.early_stop],
                    class_weight=deception_class_weight_dict)

Epoch 1/50
285/285 [==============================] - 5s 16ms/step - loss: 0.6948 - accuracy: 0.7099 - f1_m: 0.7181 - recall_m: 0.7362 - precision_m: 0.7057 - val_loss: 0.6922 - val_accuracy: 0.9507 - val_f1_m: 0.9747 - val_recall_m: 1.0000 - val_precision_m: 0.9514
Epoch 2/50
285/285 [==============================] - 4s 14ms/step - loss: 0.6941 - accuracy: 0.8901 - f1_m: 0.9223 - recall_m: 0.9344 - precision_m: 0.9417 - val_loss: 0.6921 - val_accuracy: 0.6983 - val_f1_m: 0.8060 - val_recall_m: 0.7107 - val_precision_m: 0.9384
Epoch 3/50
285/285 [==============================] - 4s 15ms/step - loss: 0.6933 - accuracy: 0.8364 - f1_m: 0.8992 - recall_m: 0.8718 - precision_m: 0.9538 - val_loss: 0.6924 - val_accuracy: 0.4771 - val_f1_m: 0.6175 - val_recall_m: 0.4642 - val_precision_m: 0.9439
Epoch 4/50
285/285 [==============================] - 4s 15ms/step - loss: 0.6846 - accuracy: 0.7085 - f1_m: 0.8048 - recall_m: 0.7210 - precision_m: 0.9656 - val_loss: 0.6282 - val_accuracy: 0.6350 

Epoch 32/50
285/285 [==============================] - 4s 15ms/step - loss: 0.3227 - accuracy: 0.8614 - f1_m: 0.9199 - recall_m: 0.8617 - precision_m: 0.9912 - val_loss: 0.9624 - val_accuracy: 0.6091 - val_f1_m: 0.7479 - val_recall_m: 0.6229 - val_precision_m: 0.9477
Epoch 33/50
285/285 [==============================] - 4s 15ms/step - loss: 0.2987 - accuracy: 0.8675 - f1_m: 0.9235 - recall_m: 0.8666 - precision_m: 0.9931 - val_loss: 0.4801 - val_accuracy: 0.8311 - val_f1_m: 0.9025 - val_recall_m: 0.8599 - val_precision_m: 0.9543
Epoch 34/50
285/285 [==============================] - 4s 16ms/step - loss: 0.2925 - accuracy: 0.8713 - f1_m: 0.9260 - recall_m: 0.8709 - precision_m: 0.9925 - val_loss: 1.0023 - val_accuracy: 0.5796 - val_f1_m: 0.7221 - val_recall_m: 0.5859 - val_precision_m: 0.9541
Epoch 35/50
285/285 [==============================] - 4s 15ms/step - loss: 0.2869 - accuracy: 0.8719 - f1_m: 0.9266 - recall_m: 0.8708 - precision_m: 0.9943 - val_loss: 0.5556 - val_accuracy: 0.7

In [23]:
deception_pred = deception_model.predict(X_train)
deception_pred_test = deception_model.predict(X_test)
deception_pred_test_round = deception_pred_test.round()
precision_recall_fscore_support(y_test_deception, deception_pred_test_round, average='macro')

(0.5024627453789942, 0.5056784392758028, 0.49471381430339223, None)

In [24]:
# deception_pred_test

In [25]:
# unique, counts = np.unique(deception_pred_test_round, return_counts=True)

In [26]:
# print(unique, counts)

In [27]:
# deception_model.history.history['val_f1_m']

### One hot encodings

In [28]:
# Train encodings
pred_df_arr_full = []
pred_df_arr = []
for i in range(0, len(gamemove_pred)):
    pred_obj_1 = {}
    pred_obj_1['gamemove'] = gamemove_pred[i][0]
    pred_obj_1['reasoning'] = reasoning_pred[i][0]
    pred_obj_1['shareinfo'] = shareinfo_pred[i][0]
    pred_df_arr.append(pred_obj_1)
    
    pred_obj_2 = pred_obj_1.copy()
    pred_obj_2['rapport'] = rapport_pred[i][0]
    pred_df_arr_full.append(pred_obj_2)
    
pred_df_full = pd.DataFrame(pred_df_arr_full)
pred_df = pd.DataFrame(pred_df_arr)

In [29]:
# Test encodings
pred_test_df_arr_full = []
pred_test_df_arr = []

for i in range(0, len(gamemove_pred_test)):
    pred_obj_1 = {}
    pred_obj_1['gamemove'] = gamemove_pred_test[i][0]
    pred_obj_1['reasoning'] = reasoning_pred_test[i][0]
    pred_obj_1['shareinfo'] = shareinfo_pred_test[i][0]
    pred_test_df_arr.append(pred_obj_1)
    
    pred_obj_2 = pred_obj_1.copy()
    pred_obj_2['rapport'] = rapport_pred_test[i][0]
    pred_test_df_arr_full.append(pred_obj_2)
    
pred_test_df_full = pd.DataFrame(pred_test_df_arr_full)
pred_test_df = pd.DataFrame(pred_test_df_arr)

### Joint Model with one hot encoding 

In [30]:
print('Joint full model with one hot encoding, predicting deception')
joint_full_model = models_nn.create_joint_model(pred_df_full)
joint_full_model.summary()
history = joint_full_model.fit(x=pred_df_full, 
                               y=y_train_deception, 
                               epochs=32, 
                               batch_size=64, 
                               validation_data=(pred_test_df_full,y_test_deception), 
#                                callbacks=[models_nn.callback], 
                               class_weight=deception_class_weight_dict)

Joint full model with one hot encoding, predicting deception
Model: "functional_13"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 4)]               0         
_________________________________________________________________
dense_12 (Dense)             (None, 2)                 10        
_________________________________________________________________
dense_13 (Dense)             (None, 4)                 12        
_________________________________________________________________
dense_14 (Dense)             (None, 1)                 5         
Total params: 27
Trainable params: 27
Non-trainable params: 0
_________________________________________________________________
Epoch 1/32
143/143 [==============================] - 0s 3ms/step - loss: 0.6998 - acc: 0.1139 - f1_m: 0.1307 - precision_m: 0.8314 - recall_m: 0.0727 - val_loss: 0.7386 - val_acc: 0.1895 - val_

143/143 [==============================] - 0s 2ms/step - loss: 0.6920 - acc: 0.5958 - f1_m: 0.7374 - precision_m: 0.9517 - recall_m: 0.6072 - val_loss: 0.6952 - val_acc: 0.6337 - val_f1_m: 0.7677 - val_precision_m: 0.9565 - val_recall_m: 0.6441
Epoch 30/32
143/143 [==============================] - 0s 2ms/step - loss: 0.6919 - acc: 0.6874 - f1_m: 0.8096 - precision_m: 0.9541 - recall_m: 0.7061 - val_loss: 0.6992 - val_acc: 0.5871 - val_f1_m: 0.7301 - val_precision_m: 0.9553 - val_recall_m: 0.5941
Epoch 31/32
143/143 [==============================] - 0s 2ms/step - loss: 0.6919 - acc: 0.6739 - f1_m: 0.7999 - precision_m: 0.9541 - recall_m: 0.6914 - val_loss: 0.6970 - val_acc: 0.6086 - val_f1_m: 0.7477 - val_precision_m: 0.9561 - val_recall_m: 0.6169
Epoch 32/32
143/143 [==============================] - 0s 2ms/step - loss: 0.6920 - acc: 0.6722 - f1_m: 0.7979 - precision_m: 0.9528 - recall_m: 0.6899 - val_loss: 0.7004 - val_acc: 0.5796 - val_f1_m: 0.7231 - val_precision_m: 0.9585 - val_r

In [31]:
joint_predict = joint_full_model.predict(pred_test_df_full)
# joint_predict_round = []
# for a in joint_predict:
#     joint_predict_round.append(np.argmax(a))
joint_predict_round = joint_predict.round()
precision_recall_fscore_support(y_test_deception, np.array(joint_predict_round), average='macro')

(0.5072350867162188, 0.5376263710849742, 0.4143173199504716, None)

In [32]:
uni, cnt = np.unique(joint_predict_round, return_counts=True)
print(uni, cnt)

[0. 1.] [ 954 1320]


In [33]:
print('Joint full model with one hot encoding, predicting rapport')
joint_full_model = models_nn.create_joint_model(pred_df)
joint_full_model.summary()
history = joint_full_model.fit(x=pred_df, 
                               y=y_train_rapport, 
                               epochs=32, 
                               batch_size=64, 
                               validation_data=(pred_test_df,y_test_rapport), 
#                                callbacks=[models_nn.callback], 
                               class_weight=rapport_class_weight_dict)

Joint full model with one hot encoding, predicting rapport
Model: "functional_15"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 3)]               0         
_________________________________________________________________
dense_15 (Dense)             (None, 2)                 8         
_________________________________________________________________
dense_16 (Dense)             (None, 4)                 12        
_________________________________________________________________
dense_17 (Dense)             (None, 1)                 5         
Total params: 25
Trainable params: 25
Non-trainable params: 0
_________________________________________________________________
Epoch 1/32
143/143 [==============================] - 0s 3ms/step - loss: 0.6868 - acc: 0.8597 - f1_m: 0.9238 - precision_m: 0.8668 - recall_m: 0.9904 - val_loss: 0.6593 - val_acc: 0.8351 - val_f1

143/143 [==============================] - 0s 2ms/step - loss: 0.6219 - acc: 0.6732 - f1_m: 0.7792 - precision_m: 0.9245 - recall_m: 0.6765 - val_loss: 0.7006 - val_acc: 0.5611 - val_f1_m: 0.6961 - val_precision_m: 0.8719 - val_recall_m: 0.5830
Epoch 30/32
143/143 [==============================] - 0s 2ms/step - loss: 0.6213 - acc: 0.6785 - f1_m: 0.7839 - precision_m: 0.9242 - recall_m: 0.6835 - val_loss: 0.6891 - val_acc: 0.5721 - val_f1_m: 0.7064 - val_precision_m: 0.8741 - val_recall_m: 0.5967
Epoch 31/32
143/143 [==============================] - 0s 2ms/step - loss: 0.6210 - acc: 0.6839 - f1_m: 0.7861 - precision_m: 0.9205 - recall_m: 0.6889 - val_loss: 0.7021 - val_acc: 0.5616 - val_f1_m: 0.6961 - val_precision_m: 0.8730 - val_recall_m: 0.5825
Epoch 32/32
143/143 [==============================] - 0s 2ms/step - loss: 0.6209 - acc: 0.6715 - f1_m: 0.7764 - precision_m: 0.9251 - recall_m: 0.6721 - val_loss: 0.6834 - val_acc: 0.5796 - val_f1_m: 0.7132 - val_precision_m: 0.8745 - val_r

In [34]:
joint_predict = joint_full_model.predict(pred_test_df)
# joint_predict_round = []
# for a in joint_predict:
#     joint_predict_round.append(np.argmax(a))
joint_predict_round = joint_predict.round()
precision_recall_fscore_support(y_test_rapport, np.array(joint_predict_round), average='macro')

(0.504031252396192, 0.5086047829809814, 0.4583151022849027, None)

In [35]:
uni, cnt = np.unique(joint_predict_round, return_counts=True)
print(uni, cnt)

[0. 1.] [ 905 1369]


## Weighted against Throughput, WorkTime, PC Agreement & Text Length

In [36]:
# Train weighted encodings
pred_df_full_throughput, pred_df_throughput, pred_df_full_worktime, pred_df_worktime, pred_df_full_agreement, pred_df_agreement, pred_df_full_textlength, pred_df_textlength, pred_df_full_special, pred_df_special = metadata_options.construct_weighted_dataframe(indices_train, df_throughput, df_worktime, df_agreement, df_textlength, df_special, pred_df, pred_df_full)

In [37]:
# Test weighted encodings
pred_df_full_throughput_test, pred_df_throughput_test, pred_df_full_worktime_test, pred_df_worktime_test, pred_df_full_agreement_test, pred_df_agreement_test, pred_df_full_textlength_test, pred_df_textlength_test, pred_df_full_special_test, pred_df_special_test = metadata_options.construct_weighted_dataframe(indices_test, df_throughput, df_worktime, df_agreement, df_textlength, df_special, pred_test_df, pred_test_df_full)

## Throughput only

### Joint Model with one hot encoding 

In [38]:
print('Joint full model with one hot encoding, predicting deception, weighted by throughput')
joint_full_model = models_nn.create_joint_model(pred_df_full_throughput)
joint_full_model.summary()
history = joint_full_model.fit(x=pred_df_full_throughput, 
                               y=y_train_deception, 
                               epochs=32, 
                               batch_size=64, 
                               validation_data=(pred_df_full_throughput_test,y_test_deception), 
#                                callbacks=[models_nn.callback], 
                               class_weight=deception_class_weight_dict)

Joint full model with one hot encoding, predicting deception, weighted by throughput
Model: "functional_17"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 4)]               0         
_________________________________________________________________
dense_18 (Dense)             (None, 2)                 10        
_________________________________________________________________
dense_19 (Dense)             (None, 4)                 12        
_________________________________________________________________
dense_20 (Dense)             (None, 1)                 5         
Total params: 27
Trainable params: 27
Non-trainable params: 0
_________________________________________________________________
Epoch 1/32
143/143 [==============================] - 0s 3ms/step - loss: 0.6987 - acc: 0.9267 - f1_m: 0.9608 - precision_m: 0.9495 - recall_m: 0.9734 - val_loss: 0.6518 

143/143 [==============================] - 0s 2ms/step - loss: 0.6932 - acc: 0.3257 - f1_m: 0.4606 - precision_m: 0.9515 - recall_m: 0.3081 - val_loss: 0.6913 - val_acc: 0.3817 - val_f1_m: 0.5277 - val_precision_m: 0.9534 - val_recall_m: 0.3677
Epoch 30/32
143/143 [==============================] - 0s 2ms/step - loss: 0.6933 - acc: 0.5199 - f1_m: 0.6451 - precision_m: 0.9533 - recall_m: 0.5229 - val_loss: 0.6918 - val_acc: 0.3527 - val_f1_m: 0.4931 - val_precision_m: 0.9546 - val_recall_m: 0.3354
Epoch 31/32
143/143 [==============================] - 0s 2ms/step - loss: 0.6932 - acc: 0.3573 - f1_m: 0.4951 - precision_m: 0.9460 - recall_m: 0.3397 - val_loss: 0.6923 - val_acc: 0.3443 - val_f1_m: 0.4832 - val_precision_m: 0.9531 - val_recall_m: 0.3266
Epoch 32/32
143/143 [==============================] - 0s 2ms/step - loss: 0.6932 - acc: 0.3472 - f1_m: 0.4846 - precision_m: 0.9540 - recall_m: 0.3292 - val_loss: 0.6917 - val_acc: 0.3615 - val_f1_m: 0.5046 - val_precision_m: 0.9530 - val_r

In [39]:
joint_predict = joint_full_model.predict(pred_df_full_throughput_test)
# joint_predict_round = []
# for a in joint_predict:
#     joint_predict_round.append(np.argmax(a))
joint_predict_round = joint_predict.round()
precision_recall_fscore_support(y_test_deception, np.array(joint_predict_round), average='macro')

(0.502451266087505, 0.5118111536936699, 0.30077828526509814, None)

In [40]:
uni, cnt = np.unique(joint_predict_round, return_counts=True)
print(uni, cnt)

[0. 1.] [1492  782]


In [41]:
y_test_rapport = np.asarray(y_test_rapport)
y_train_rapport = np.asarray(y_train_rapport)

In [42]:
print('Joint full model with one hot encoding, predicting rapport, weighted by throughput')
joint_full_model = models_nn.create_joint_model(pred_df_throughput)
joint_full_model.summary()
history = joint_full_model.fit(x=pred_df_throughput, 
                               y=y_train_rapport, 
                               epochs=32, 
                               batch_size=64,
                               validation_data=(pred_df_throughput_test,y_test_rapport), 
#                                callbacks=[models_nn.callback], 
                               class_weight=rapport_class_weight_dict)

Joint full model with one hot encoding, predicting rapport, weighted by throughput
Model: "functional_19"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         [(None, 3)]               0         
_________________________________________________________________
dense_21 (Dense)             (None, 2)                 8         
_________________________________________________________________
dense_22 (Dense)             (None, 4)                 12        
_________________________________________________________________
dense_23 (Dense)             (None, 1)                 5         
Total params: 25
Trainable params: 25
Non-trainable params: 0
_________________________________________________________________
Epoch 1/32
143/143 [==============================] - 0s 3ms/step - loss: 0.7012 - acc: 0.5016 - f1_m: 0.6195 - precision_m: 0.8557 - recall_m: 0.5063 - val_loss: 0.6932 - 

143/143 [==============================] - 0s 2ms/step - loss: 0.6883 - acc: 0.6344 - f1_m: 0.7499 - precision_m: 0.8863 - recall_m: 0.6573 - val_loss: 0.6958 - val_acc: 0.5427 - val_f1_m: 0.6699 - val_precision_m: 0.8865 - val_recall_m: 0.5413
Epoch 30/32
143/143 [==============================] - 0s 2ms/step - loss: 0.6882 - acc: 0.5881 - f1_m: 0.7060 - precision_m: 0.8841 - recall_m: 0.5963 - val_loss: 0.6827 - val_acc: 0.6420 - val_f1_m: 0.7630 - val_precision_m: 0.8857 - val_recall_m: 0.6727
Epoch 31/32
143/143 [==============================] - 0s 2ms/step - loss: 0.6882 - acc: 0.5768 - f1_m: 0.6958 - precision_m: 0.8804 - recall_m: 0.5850 - val_loss: 0.6852 - val_acc: 0.6275 - val_f1_m: 0.7509 - val_precision_m: 0.8849 - val_recall_m: 0.6547
Epoch 32/32
143/143 [==============================] - 0s 2ms/step - loss: 0.6881 - acc: 0.5989 - f1_m: 0.7167 - precision_m: 0.8819 - recall_m: 0.6121 - val_loss: 0.6960 - val_acc: 0.5457 - val_f1_m: 0.6731 - val_precision_m: 0.8862 - val_r

In [43]:
joint_predict = joint_full_model.predict(pred_df_throughput_test)
# joint_predict_round = []
# for a in joint_predict:
#     joint_predict_round.append(np.argmax(a))
joint_predict_round = joint_predict.round()
precision_recall_fscore_support(y_test_rapport, np.array(joint_predict_round), average='macro')

(0.5193918868487833, 0.5429713679270476, 0.45566709018362167, None)

In [44]:
uni, cnt = np.unique(joint_predict_round, return_counts=True)
print(uni, cnt)

[0. 1.] [1056 1218]


## WorkTime only

### Joint Model with one hot encoding 

In [45]:
print('Joint full model with one hot encoding, predicting deception, weighted by worktime')
joint_full_model = models_nn.create_joint_model(pred_df_full_worktime)
joint_full_model.summary()
history = joint_full_model.fit(x=pred_df_full_worktime, 
                               y=y_train_deception, 
                               epochs=32, 
                               batch_size=64, 
                               validation_data=(pred_df_full_worktime_test,y_test_deception), 
#                                callbacks=[models_nn.callback],
                               class_weight=deception_class_weight_dict)

Joint full model with one hot encoding, predicting deception, weighted by worktime
Model: "functional_21"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         [(None, 4)]               0         
_________________________________________________________________
dense_24 (Dense)             (None, 2)                 10        
_________________________________________________________________
dense_25 (Dense)             (None, 4)                 12        
_________________________________________________________________
dense_26 (Dense)             (None, 1)                 5         
Total params: 27
Trainable params: 27
Non-trainable params: 0
_________________________________________________________________
Epoch 1/32
143/143 [==============================] - 0s 3ms/step - loss: 0.6933 - acc: 0.6531 - f1_m: 0.7572 - precision_m: 0.9516 - recall_m: 0.6688 - val_loss: 0.6902 - 

143/143 [==============================] - 0s 2ms/step - loss: 0.6932 - acc: 0.2576 - f1_m: 0.3616 - precision_m: 0.9525 - recall_m: 0.2317 - val_loss: 0.6959 - val_acc: 0.1983 - val_f1_m: 0.2788 - val_precision_m: 0.9475 - val_recall_m: 0.1650
Epoch 30/32
143/143 [==============================] - 0s 2ms/step - loss: 0.6933 - acc: 0.0613 - f1_m: 0.0236 - precision_m: 0.1274 - recall_m: 0.0136 - val_loss: 0.7004 - val_acc: 0.0523 - val_f1_m: 0.0063 - val_precision_m: 0.1667 - val_recall_m: 0.0032
Epoch 31/32
143/143 [==============================] - 0s 2ms/step - loss: 0.6930 - acc: 0.6060 - f1_m: 0.7132 - precision_m: 0.9348 - recall_m: 0.6164 - val_loss: 0.6998 - val_acc: 0.3135 - val_f1_m: 0.4448 - val_precision_m: 0.9501 - val_recall_m: 0.2924
Epoch 32/32
143/143 [==============================] - 0s 2ms/step - loss: 0.6931 - acc: 0.5548 - f1_m: 0.6783 - precision_m: 0.9562 - recall_m: 0.5608 - val_loss: 0.7007 - val_acc: 0.2973 - val_f1_m: 0.4226 - val_precision_m: 0.9538 - val_r

In [46]:
joint_predict = joint_full_model.predict(pred_df_full_worktime_test)
# joint_predict_round = []
# for a in joint_predict:
#     joint_predict_round.append(np.argmax(a))
joint_predict_round = joint_predict.round()
precision_recall_fscore_support(y_test_deception, np.array(joint_predict_round), average='macro')

(0.5018091682691933, 0.5076772499008855, 0.26005502775822975, None)

In [47]:
uni, cnt = np.unique(joint_predict_round, return_counts=True)
print(uni, cnt)

[0. 1.] [1652  622]


In [48]:
print('Joint full model with one hot encoding, predicting rapport, weighted by worktime')
joint_full_model = models_nn.create_joint_model(pred_df_worktime)
joint_full_model.summary()
history = joint_full_model.fit(x=pred_df_worktime, 
                               y=y_train_rapport, 
                               epochs=32, 
                               batch_size=64,
                               validation_data=(pred_df_worktime_test,y_test_rapport), 
#                                callbacks=[models_nn.callback], 
                               class_weight=rapport_class_weight_dict)

Joint full model with one hot encoding, predicting rapport, weighted by worktime
Model: "functional_23"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_6 (InputLayer)         [(None, 3)]               0         
_________________________________________________________________
dense_27 (Dense)             (None, 2)                 8         
_________________________________________________________________
dense_28 (Dense)             (None, 4)                 12        
_________________________________________________________________
dense_29 (Dense)             (None, 1)                 5         
Total params: 25
Trainable params: 25
Non-trainable params: 0
_________________________________________________________________
Epoch 1/32
143/143 [==============================] - 0s 3ms/step - loss: 0.6944 - acc: 0.6524 - f1_m: 0.7606 - precision_m: 0.8353 - recall_m: 0.7333 - val_loss: 0.6905 - va

143/143 [==============================] - 0s 2ms/step - loss: 0.6689 - acc: 0.5605 - f1_m: 0.6796 - precision_m: 0.9035 - recall_m: 0.5479 - val_loss: 0.6873 - val_acc: 0.5361 - val_f1_m: 0.6619 - val_precision_m: 0.8978 - val_recall_m: 0.5275
Epoch 30/32
143/143 [==============================] - 0s 2ms/step - loss: 0.6686 - acc: 0.5744 - f1_m: 0.6941 - precision_m: 0.9024 - recall_m: 0.5675 - val_loss: 0.6833 - val_acc: 0.5405 - val_f1_m: 0.6664 - val_precision_m: 0.8964 - val_recall_m: 0.5334
Epoch 31/32
143/143 [==============================] - 0s 2ms/step - loss: 0.6685 - acc: 0.5844 - f1_m: 0.7061 - precision_m: 0.9028 - recall_m: 0.5833 - val_loss: 0.6902 - val_acc: 0.5334 - val_f1_m: 0.6592 - val_precision_m: 0.8980 - val_recall_m: 0.5241
Epoch 32/32
143/143 [==============================] - 0s 2ms/step - loss: 0.6682 - acc: 0.5779 - f1_m: 0.6973 - precision_m: 0.9020 - recall_m: 0.5721 - val_loss: 0.6945 - val_acc: 0.5286 - val_f1_m: 0.6540 - val_precision_m: 0.8981 - val_r

In [49]:
joint_predict = joint_full_model.predict(pred_df_worktime_test)
# joint_predict_round = []
# for a in joint_predict:
#     joint_predict_round.append(np.argmax(a))
joint_predict_round = joint_predict.round()
precision_recall_fscore_support(y_test_rapport, np.array(joint_predict_round), average='macro')

(0.5266261478588385, 0.5593014180792615, 0.45202060164870733, None)

In [50]:
uni, cnt = np.unique(joint_predict_round, return_counts=True)
print(uni, cnt)

[0. 1.] [1131 1143]


## PC Agreement only

### Joint Model with one hot encoding 

In [51]:
print('Joint full model with one hot encoding, predicting deception, weighted by PC Agreement')
joint_full_model = models_nn.create_joint_model(pred_df_full_agreement)
joint_full_model.summary()
history = joint_full_model.fit(x=pred_df_full_agreement, 
                               y=y_train_deception, 
                               epochs=32, 
                               batch_size=64, 
                               validation_data=(pred_df_full_agreement_test,y_test_deception), 
#                                callbacks=[models_nn.callback],
                               class_weight=deception_class_weight_dict)

Joint full model with one hot encoding, predicting deception, weighted by PC Agreement
Model: "functional_25"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_7 (InputLayer)         [(None, 4)]               0         
_________________________________________________________________
dense_30 (Dense)             (None, 2)                 10        
_________________________________________________________________
dense_31 (Dense)             (None, 4)                 12        
_________________________________________________________________
dense_32 (Dense)             (None, 1)                 5         
Total params: 27
Trainable params: 27
Non-trainable params: 0
_________________________________________________________________
Epoch 1/32
143/143 [==============================] - 1s 4ms/step - loss: 0.6933 - acc: 0.5238 - f1_m: 0.5231 - precision_m: 0.6641 - recall_m: 0.5347 - val_loss: 0.692

143/143 [==============================] - 0s 2ms/step - loss: 0.6931 - acc: 0.0941 - f1_m: 0.0714 - precision_m: 0.8101 - recall_m: 0.0511 - val_loss: 0.6939 - val_acc: 0.0989 - val_f1_m: 0.1004 - val_precision_m: 0.9090 - val_recall_m: 0.0541
Epoch 30/32
143/143 [==============================] - 0s 2ms/step - loss: 0.6931 - acc: 0.0947 - f1_m: 0.0987 - precision_m: 0.8831 - recall_m: 0.0540 - val_loss: 0.6937 - val_acc: 0.1077 - val_f1_m: 0.1170 - val_precision_m: 0.9311 - val_recall_m: 0.0638
Epoch 31/32
143/143 [==============================] - 0s 2ms/step - loss: 0.6931 - acc: 0.0735 - f1_m: 0.0516 - precision_m: 0.7348 - recall_m: 0.0271 - val_loss: 0.6934 - val_acc: 0.0849 - val_f1_m: 0.0714 - val_precision_m: 0.8472 - val_recall_m: 0.0379
Epoch 32/32
143/143 [==============================] - 0s 2ms/step - loss: 0.6930 - acc: 0.2245 - f1_m: 0.2191 - precision_m: 0.8526 - recall_m: 0.2003 - val_loss: 0.6909 - val_acc: 0.9507 - val_f1_m: 0.9745 - val_precision_m: 0.9506 - val_r

In [52]:
joint_predict = joint_full_model.predict(pred_df_full_throughput_test)
# joint_predict_round = []
# for a in joint_predict:
#     joint_predict_round.append(np.argmax(a))
joint_predict_round = joint_predict.round()
precision_recall_fscore_support(y_test_rapport, np.array(joint_predict_round), average='macro')

/home/kaiyuan/anaconda3/envs/TF2/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


(0.4355760773966579, 0.5, 0.4655699177438308, None)

In [53]:
uni, cnt = np.unique(joint_predict_round, return_counts=True)
print(uni, cnt)

[1.] [2274]


In [54]:
y_test_rapport = np.asarray(y_test_rapport)
y_train_rapport = np.asarray(y_train_rapport)

In [55]:
print('Joint full model with one hot encoding, predicting rapport, weighted by PC Agreement')
joint_full_model = models_nn.create_joint_model(pred_df_agreement)
joint_full_model.summary()
history = joint_full_model.fit(x=pred_df_agreement, 
                               y=y_train_rapport, 
                               epochs=32, 
                               batch_size=64, 
                               validation_data=(pred_df_agreement_test,y_test_rapport), 
#                                callbacks=[models_nn.callback], 
                               class_weight=rapport_class_weight_dict)

Joint full model with one hot encoding, predicting rapport, weighted by PC Agreement
Model: "functional_27"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_8 (InputLayer)         [(None, 3)]               0         
_________________________________________________________________
dense_33 (Dense)             (None, 2)                 8         
_________________________________________________________________
dense_34 (Dense)             (None, 4)                 12        
_________________________________________________________________
dense_35 (Dense)             (None, 1)                 5         
Total params: 25
Trainable params: 25
Non-trainable params: 0
_________________________________________________________________
Epoch 1/32
143/143 [==============================] - 0s 3ms/step - loss: 0.6787 - acc: 0.8203 - f1_m: 0.8969 - precision_m: 0.8817 - recall_m: 0.9157 - val_loss: 0.6403 

143/143 [==============================] - 0s 2ms/step - loss: 0.6409 - acc: 0.5911 - f1_m: 0.7007 - precision_m: 0.9099 - recall_m: 0.5737 - val_loss: 0.6925 - val_acc: 0.5211 - val_f1_m: 0.6455 - val_precision_m: 0.8993 - val_recall_m: 0.5069
Epoch 30/32
143/143 [==============================] - 0s 2ms/step - loss: 0.6409 - acc: 0.5799 - f1_m: 0.6950 - precision_m: 0.9184 - recall_m: 0.5627 - val_loss: 0.6659 - val_acc: 0.5532 - val_f1_m: 0.6804 - val_precision_m: 0.8944 - val_recall_m: 0.5529
Epoch 31/32
143/143 [==============================] - 0s 2ms/step - loss: 0.6408 - acc: 0.5860 - f1_m: 0.6979 - precision_m: 0.9148 - recall_m: 0.5685 - val_loss: 0.6730 - val_acc: 0.5475 - val_f1_m: 0.6735 - val_precision_m: 0.8969 - val_recall_m: 0.5434
Epoch 32/32
143/143 [==============================] - 0s 2ms/step - loss: 0.6411 - acc: 0.5841 - f1_m: 0.7010 - precision_m: 0.9199 - recall_m: 0.5707 - val_loss: 0.6757 - val_acc: 0.5444 - val_f1_m: 0.6699 - val_precision_m: 0.8977 - val_r

In [56]:
joint_predict = joint_full_model.predict(pred_df_throughput_test)
# joint_predict_round = []
# for a in joint_predict:
#     joint_predict_round.append(np.argmax(a))
joint_predict_round = joint_predict.round()
precision_recall_fscore_support(y_test_rapport, np.array(joint_predict_round), average='macro')

(0.46200224392917183, 0.42928210491133345, 0.43150836017117394, None)

In [57]:
uni, cnt = np.unique(joint_predict_round, return_counts=True)
print(uni, cnt)

[0. 1.] [ 676 1598]


## Text Length only

### Joint Model with one hot encoding 

In [58]:
print('Joint full model with one hot encoding, predicting deception, weighted by text length')
joint_full_model = models_nn.create_joint_model(pred_df_full_textlength)
joint_full_model.summary()
history = joint_full_model.fit(x=pred_df_full_textlength, 
                               y=y_train_deception, 
                               epochs=32, 
                               batch_size=64, 
                               validation_data=(pred_df_full_textlength_test,y_test_deception), 
#                                callbacks=[models_nn.callback],
                               class_weight=deception_class_weight_dict)

Joint full model with one hot encoding, predicting deception, weighted by text length
Model: "functional_29"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_9 (InputLayer)         [(None, 4)]               0         
_________________________________________________________________
dense_36 (Dense)             (None, 2)                 10        
_________________________________________________________________
dense_37 (Dense)             (None, 4)                 12        
_________________________________________________________________
dense_38 (Dense)             (None, 1)                 5         
Total params: 27
Trainable params: 27
Non-trainable params: 0
_________________________________________________________________
Epoch 1/32
143/143 [==============================] - 0s 3ms/step - loss: 0.6981 - acc: 0.8591 - f1_m: 0.9222 - precision_m: 0.9511 - recall_m: 0.8990 - val_loss: 0.6700

143/143 [==============================] - 0s 2ms/step - loss: 0.6931 - acc: 0.3511 - f1_m: 0.3766 - precision_m: 0.4668 - recall_m: 0.3370 - val_loss: 0.6901 - val_acc: 0.7801 - val_f1_m: 0.8744 - val_precision_m: 0.9523 - val_recall_m: 0.8099
Epoch 30/32
143/143 [==============================] - 0s 2ms/step - loss: 0.6930 - acc: 0.7491 - f1_m: 0.8509 - precision_m: 0.9521 - recall_m: 0.7757 - val_loss: 0.6888 - val_acc: 0.7924 - val_f1_m: 0.8824 - val_precision_m: 0.9512 - val_recall_m: 0.8245
Epoch 31/32
143/143 [==============================] - 0s 2ms/step - loss: 0.6930 - acc: 0.7973 - f1_m: 0.8855 - precision_m: 0.9534 - recall_m: 0.8291 - val_loss: 0.6867 - val_acc: 0.8259 - val_f1_m: 0.9039 - val_precision_m: 0.9524 - val_recall_m: 0.8611
Epoch 32/32
143/143 [==============================] - 0s 2ms/step - loss: 0.6929 - acc: 0.8344 - f1_m: 0.9086 - precision_m: 0.9530 - recall_m: 0.8700 - val_loss: 0.6885 - val_acc: 0.7775 - val_f1_m: 0.8728 - val_precision_m: 0.9523 - val_r

In [59]:
joint_predict = joint_full_model.predict(pred_df_full_textlength_test)
# joint_predict_round = []
# for a in joint_predict:
#     joint_predict_round.append(np.argmax(a))
joint_predict_round = joint_predict.round()
precision_recall_fscore_support(y_test_deception, np.array(joint_predict_round), average='macro')

(0.50313185401806, 0.5104731069115898, 0.4799762821126997, None)

In [60]:
uni, cnt = np.unique(joint_predict_round, return_counts=True)
print(uni, cnt)

[0. 1.] [ 442 1832]


In [61]:
print('Joint full model with one hot encoding, predicting rapport, weighted by text length')
joint_full_model = models_nn.create_joint_model(pred_df_textlength)
joint_full_model.summary()
history = joint_full_model.fit(x=pred_df_textlength, 
                               y=y_train_rapport, 
                               epochs=32, 
                               batch_size=64, 
                               validation_data=(pred_df_textlength_test,y_test_rapport), 
#                                callbacks=[models_nn.callback], 
                               class_weight=rapport_class_weight_dict)

Joint full model with one hot encoding, predicting rapport, weighted by text length
Model: "functional_31"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_10 (InputLayer)        [(None, 3)]               0         
_________________________________________________________________
dense_39 (Dense)             (None, 2)                 8         
_________________________________________________________________
dense_40 (Dense)             (None, 4)                 12        
_________________________________________________________________
dense_41 (Dense)             (None, 1)                 5         
Total params: 25
Trainable params: 25
Non-trainable params: 0
_________________________________________________________________
Epoch 1/32
143/143 [==============================] - 0s 3ms/step - loss: 0.6930 - acc: 0.3072 - f1_m: 0.3011 - precision_m: 0.5425 - recall_m: 0.2340 - val_loss: 0.6937 -

143/143 [==============================] - 0s 2ms/step - loss: 0.6618 - acc: 0.7267 - f1_m: 0.8281 - precision_m: 0.8955 - recall_m: 0.7734 - val_loss: 0.6724 - val_acc: 0.6895 - val_f1_m: 0.8045 - val_precision_m: 0.8821 - val_recall_m: 0.7418
Epoch 30/32
143/143 [==============================] - 0s 2ms/step - loss: 0.6612 - acc: 0.7177 - f1_m: 0.8202 - precision_m: 0.8992 - recall_m: 0.7569 - val_loss: 0.6804 - val_acc: 0.6746 - val_f1_m: 0.7940 - val_precision_m: 0.8775 - val_recall_m: 0.7276
Epoch 31/32
143/143 [==============================] - 0s 2ms/step - loss: 0.6599 - acc: 0.7363 - f1_m: 0.8351 - precision_m: 0.8959 - recall_m: 0.7845 - val_loss: 0.6986 - val_acc: 0.6469 - val_f1_m: 0.7705 - val_precision_m: 0.8819 - val_recall_m: 0.6874
Epoch 32/32
143/143 [==============================] - 0s 2ms/step - loss: 0.6591 - acc: 0.7222 - f1_m: 0.8238 - precision_m: 0.8963 - recall_m: 0.7661 - val_loss: 0.6853 - val_acc: 0.6653 - val_f1_m: 0.7860 - val_precision_m: 0.8802 - val_r

In [62]:
joint_predict = joint_full_model.predict(pred_df_textlength_test)
# joint_predict_round = []
# for a in joint_predict:
#     joint_predict_round.append(np.argmax(a))
joint_predict_round = joint_predict.round()
precision_recall_fscore_support(y_test_rapport, np.array(joint_predict_round), average='macro')

(0.5170344955489614, 0.5316487863370967, 0.5002597588617623, None)

In [63]:
uni, cnt = np.unique(joint_predict_round, return_counts=True)
print(uni, cnt)

[0. 1.] [ 674 1600]


## Other special options (either SP1, SP2, SP3, RAND_UNI, or RAND_NORM)

### Joint Model with one hot encoding 

In [64]:
print('Joint full model with one hot encoding, predicting deception, weighted by special option')
joint_full_model = models_nn.create_joint_model(pred_df_full_special)
joint_full_model.summary()
history = joint_full_model.fit(x=pred_df_full_special, 
                               y=y_train_deception, 
                               epochs=32, 
                               batch_size=64, 
                               validation_data=(pred_df_full_special_test,y_test_deception), 
#                                callbacks=[models_nn.callback],
                               class_weight=deception_class_weight_dict)

Joint full model with one hot encoding, predicting deception, weighted by special option
Model: "functional_33"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_11 (InputLayer)        [(None, 4)]               0         
_________________________________________________________________
dense_42 (Dense)             (None, 2)                 10        
_________________________________________________________________
dense_43 (Dense)             (None, 4)                 12        
_________________________________________________________________
dense_44 (Dense)             (None, 1)                 5         
Total params: 27
Trainable params: 27
Non-trainable params: 0
_________________________________________________________________
Epoch 1/32
143/143 [==============================] - 0s 3ms/step - loss: 0.6931 - acc: 0.4173 - f1_m: 0.4736 - precision_m: 0.9560 - recall_m: 0.4082 - val_loss: 0.6

143/143 [==============================] - 0s 2ms/step - loss: 0.6923 - acc: 0.5918 - f1_m: 0.7350 - precision_m: 0.9548 - recall_m: 0.6007 - val_loss: 0.6911 - val_acc: 0.5545 - val_f1_m: 0.7019 - val_precision_m: 0.9526 - val_recall_m: 0.5591
Epoch 30/32
143/143 [==============================] - 0s 2ms/step - loss: 0.6922 - acc: 0.5396 - f1_m: 0.6878 - precision_m: 0.9520 - recall_m: 0.5424 - val_loss: 0.6889 - val_acc: 0.5818 - val_f1_m: 0.7252 - val_precision_m: 0.9518 - val_recall_m: 0.5888
Epoch 31/32
143/143 [==============================] - 0s 2ms/step - loss: 0.6923 - acc: 0.5378 - f1_m: 0.6847 - precision_m: 0.9549 - recall_m: 0.5376 - val_loss: 0.6937 - val_acc: 0.5255 - val_f1_m: 0.6761 - val_precision_m: 0.9516 - val_recall_m: 0.5278
Epoch 32/32
143/143 [==============================] - 0s 2ms/step - loss: 0.6922 - acc: 0.5542 - f1_m: 0.7001 - precision_m: 0.9542 - recall_m: 0.5560 - val_loss: 0.6888 - val_acc: 0.5818 - val_f1_m: 0.7252 - val_precision_m: 0.9518 - val_r

In [65]:
joint_predict = joint_full_model.predict(pred_df_full_special_test)
# joint_predict_round = []
# for a in joint_predict:
#     joint_predict_round.append(np.argmax(a))
joint_predict_round = joint_predict.round()
precision_recall_fscore_support(y_test_deception, np.array(joint_predict_round), average='macro')

(0.501770017532439, 0.5091515792255847, 0.41002693458848033, None)

In [66]:
uni, cnt = np.unique(joint_predict_round, return_counts=True)
print(uni, cnt)

[0. 1.] [ 935 1339]


In [67]:
y_test_rapport = np.asarray(y_test_rapport)
y_train_rapport = np.asarray(y_train_rapport)

In [68]:
print('Joint full model with one hot encoding, predicting rapport, weighted by special option')
joint_full_model = models_nn.create_joint_model(pred_df_special)
joint_full_model.summary()
history = joint_full_model.fit(x=pred_df_special, 
                               y=y_train_rapport, 
                               epochs=32, 
                               batch_size=64,
                               validation_data=(pred_df_special_test,y_test_rapport), 
#                                callbacks=[models_nn.callback], 
                               class_weight=rapport_class_weight_dict)

Joint full model with one hot encoding, predicting rapport, weighted by special option
Model: "functional_35"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_12 (InputLayer)        [(None, 3)]               0         
_________________________________________________________________
dense_45 (Dense)             (None, 2)                 8         
_________________________________________________________________
dense_46 (Dense)             (None, 4)                 12        
_________________________________________________________________
dense_47 (Dense)             (None, 1)                 5         
Total params: 25
Trainable params: 25
Non-trainable params: 0
_________________________________________________________________
Epoch 1/32
143/143 [==============================] - 0s 3ms/step - loss: 0.6979 - acc: 0.5570 - f1_m: 0.6746 - precision_m: 0.8427 - recall_m: 0.5935 - val_loss: 0.694

143/143 [==============================] - 0s 2ms/step - loss: 0.6673 - acc: 0.7534 - f1_m: 0.8473 - precision_m: 0.8924 - recall_m: 0.8082 - val_loss: 0.6852 - val_acc: 0.7212 - val_f1_m: 0.8293 - val_precision_m: 0.8762 - val_recall_m: 0.7890
Epoch 30/32
143/143 [==============================] - 0s 2ms/step - loss: 0.6671 - acc: 0.7519 - f1_m: 0.8479 - precision_m: 0.8941 - recall_m: 0.8083 - val_loss: 0.6835 - val_acc: 0.7265 - val_f1_m: 0.8331 - val_precision_m: 0.8771 - val_recall_m: 0.7950
Epoch 31/32
143/143 [==============================] - 0s 2ms/step - loss: 0.6666 - acc: 0.7546 - f1_m: 0.8490 - precision_m: 0.8943 - recall_m: 0.8105 - val_loss: 0.6902 - val_acc: 0.7115 - val_f1_m: 0.8221 - val_precision_m: 0.8771 - val_recall_m: 0.7758
Epoch 32/32
143/143 [==============================] - 0s 2ms/step - loss: 0.6668 - acc: 0.7511 - f1_m: 0.8476 - precision_m: 0.8948 - recall_m: 0.8081 - val_loss: 0.6794 - val_acc: 0.7331 - val_f1_m: 0.8380 - val_precision_m: 0.8778 - val_r

In [69]:
joint_predict = joint_full_model.predict(pred_df_special_test)
# joint_predict_round = []
# for a in joint_predict:
#     joint_predict_round.append(np.argmax(a))
joint_predict_round = joint_predict.round()
precision_recall_fscore_support(y_test_rapport, np.array(joint_predict_round), average='macro')

(0.5205979710421033, 0.5298036121309436, 0.5190172650444091, None)

In [70]:
uni, cnt = np.unique(joint_predict_round, return_counts=True)
print(uni, cnt)

[0. 1.] [ 464 1810]
